## **Setup**

In [18]:
# !pip install opencv-python
# !pip install imutils

# !pip install fastai fastcore --upgrade

# !cd fastbook
# !git pull

  Created wheel for imutils: filename=imutils-0.5.3-py3-none-any.whl size=25853 sha256=eee6cf91e03b56420cd1fe06e9d0fa99842f0f3ffccb0b10ca8fcf587c3e0fe3
  Stored in directory: /root/.cache/pip/wheels/c8/d6/0f/b0c3892b70c59f0d202f8619a449f7d14cb839a0af2f943869
Successfully built imutils


In [2]:
from fastai.vision.all import *

In [19]:
import os
import shutil
import glob
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import pytz
from zipfile import ZipFile
from tempfile import TemporaryDirectory
from PIL import Image

import cv2
import imutils
from scipy.optimize import minimize_scalar

from sklearn.metrics import f1_score, accuracy_score

# plot options
# plt.rcParams.update({'font.size': 11})
plt.style.use('fivethirtyeight')

# Functions

In [4]:
def sqdif(bingray,sq,ang):
    rsq=imutils.resize(imutils.rotate_bound(sq,ang),width=bingray.shape[0],height=bingray.shape[0])

    return np.sum(np.abs(bingray-rsq))

In [5]:
def derotate(img):
    sq=np.ones(img.shape)*255
    imsz = img.shape[0]

    # binarize by blurring then using Otsu's method
    # blur=cv2.GaussianBlur(gray,(1,1),0)
    _,bingray = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    rotang=minimize_scalar(lambda ang: sqdif(bingray,sq,ang),
                         bounds=[-45,45], method='Bounded').x

    if rotang<0:
        triang=np.tan(np.pi/4+rotang*np.pi/180)
        act_x=imsz/2*(1+triang)
        act_y=imsz/2*(1-triang)

        pts1 = np.float32([[0,act_y],[act_x,0],[imsz-act_x,imsz],[imsz,imsz-act_y]])

    else:
        triang=np.tan(np.pi/4-rotang*np.pi/180)
        act_x=imsz/2*(1-triang)
        act_y=imsz/2*(1+triang)

        pts1 = np.float32([[act_x,0],[imsz,imsz-act_y],[0,act_y],[imsz-act_x,imsz]])

    pts2 = np.float32([[0,0],[imsz,0],[0,imsz],[imsz,imsz]])
    M = cv2.getPerspectiveTransform(pts1,pts2)

    return cv2.warpPerspective(img,M,(imsz,imsz))

In [6]:
# SUDOKU SOLVING ALGORITHM BY PETER NORVIG https://norvig.com/sudoku.html
# slightly modified by Benjamin P Isaacoff on 8/29/20
# original code (pre-modification by BPI) taken from https://github.com/Adityaojas/sudoku-solver/blob/master/norvig.py

def cross(A, B):
  "Cross product of elements in A and elements in B."
  return [a+b for a in A for b in B]
    
digits   = '123456789'
rows     = 'ABCDEFGHI'
cols     = digits
squares  = cross(rows, cols)
unitlist = ([cross(rows, c) for c in cols] +
        [cross(r, cols) for r in rows] +
        [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')])
units = dict((s, [u for u in unitlist if s in u]) 
         for s in squares)
peers = dict((s, set(sum(units[s],[]))-set([s]))
         for s in squares)


def test():
  "A set of unit tests."
  assert len(squares) == 81
  assert len(unitlist) == 27
  assert np.all(len(units[s]) == 3 for s in squares)
  assert np.all(len(peers[s]) == 20 for s in squares)
  assert units['C2'] == [['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2'],
                          ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'],
                          ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']]
  assert peers['C2'] == set(['A2', 'B2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2',
                              'C1', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9',
                              'A1', 'A3', 'B1', 'B3'])
  print('All tests pass.')


def parse_grid(grid):
  """Convert grid to a dict of possible values, {square: digits}, or
  return False if a contradiction is detected."""
  ## To start, every square can be any digit; then assign values from the grid.
  values = dict([(s, digits) for s in squares])
  for s,d in list(grid_values(grid).items()):
      if d in digits and not assign(values, s, d):
          return False ## (Fail if we can't assign d to square s.)
  return values

def grid_values(grid):
  "Convert grid into a dict of {square: char} with '0' or '.' for empties."
  chars = [c for c in grid if c in digits or c in '0.']
  assert len(chars) == 81
  return dict(list(zip(squares, chars)))


def assign(values, s, d):
  """Eliminate all the other values (except d) from values[s] and propagate.
  Return values, except return False if a contradiction is detected."""
  other_values = values[s].replace(d, '')
  if np.all([eliminate(values, s, d2) for d2 in other_values]):
      return values
  else:
      return False

def eliminate(values, s, d):
  """Eliminate d from values[s]; propagate when values or places <= 2.
  Return values, except return False if a contradiction is detected."""
  if d not in values[s]:
      return values ## Already eliminated
  values[s] = values[s].replace(d,'')
  ## (1) If a square s is reduced to one value d2, then eliminate d2 from the peers.
  if len(values[s]) == 0:
    return False ## Contradiction: removed last value
  elif len(values[s]) == 1:
      d2 = values[s]
      if not np.all([eliminate(values, s2, d2) for s2 in peers[s]]):
          return False
  ## (2) If a unit u is reduced to only one place for a value d, then put it there.
  for u in units[s]:
    dplaces = [s for s in u if d in values[s]]
    if len(dplaces) == 0:
        return False ## Contradiction: no place for this value
    elif len(dplaces) == 1:
        # d can only be in one place in unit; assign it there
          if not assign(values, dplaces[0], d):
              return False
  return values

def sudoku_display(instr):
  for rowind in range(9):
    rowstr=[]
    for colind in range(9):
      rowstr.append(str(instr[9*rowind+colind]))
      if colind==2 or colind==5: rowstr.append('|')
    print(' '.join(rowstr))

    if rowind==2 or rowind==5:
      print('---------------------')
  print(' ')
    
def solve(grid):
  return search(parse_grid(grid))


def search(values):
  "Using depth-first search and propagation, try all possible values."
  if values is False:
      # print('False here 000')
      return False ## Failed earlier
  if np.all([len(values[s]) == 1 for s in squares]): 
      return values ## Solved!
  ## Chose the unfilled square s with the fewest possibilities
  n,s = min([(len(values[s]), s) for s in squares if len(values[s]) > 1])
  return some([search(assign(values.copy(), s, d)) for d in values[s]])

def some(seq):
  "Return some element of seq that is true."
  for e in seq:
      if e: return e
  return False


#solve("400000805030000000000700000020000060000080400000010000000603070500200000104000000")
#solve([0, 0, 0, 0, 0, 0, 8, 1, 8, 0, 0, 0, 2, 3, 0, 0, 0, 6, 0, 0, 5, 7, 0, 0, 1, 0, 7, 0, 9, 6, 0, 0, 0, 0, 0, 9, 0, 7, 0, 4, 0, 1, 0, 0, 0, 0, 0, 8, 1, 0, 4, 0, 6, 0, 0, 2, 4, 0, 0, 8, 0, 0, 4, 5, 0, 0, 9, 3, 5, 0, 0, 0, 0, 0, 0])

#parse_grid("4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......")

# Load Sudoku Data

In [7]:
home_dir='/storage/BPI/Data/Sudoku/All_Data/'

# this can take a bit (maybe 30s)
# All_Data is the home directory for the images, from which training and validation splits will be taken
if not os.path.exists(home_dir):
    os.mkdir(home_dir)
    shutil.unpack_archive('/storage/BPI/Data/Sudoku/test.tar',home_dir)  

In [21]:
imgs=glob.glob(home_dir+'test/images/*.png')
print(len(imgs))
imgs=imgs[3500:3700]

5001


In [22]:
learn=load_learner('/storage/BPI/Data/Sudoku/200909_1625_fullgennums.pkl')

In [23]:
labeldict={0:'1',1:'2',2:'3',3:'4',4:'5',5:'6',6:'7',7:'8',8:'9',9:'.'}

tmpdir='imgs4solv/'
# this can take a bit (maybe 30s)
# All_Data is the home directory for the images, from which training and validation splits will be taken
if not os.path.exists(f'{home_dir}{tmpdir}'):
  os.mkdir(f'{home_dir}{tmpdir}')

In [24]:
def extractNsolve(impath):
  imid=os.path.basename(impath)[:-4]  

  img=cv2.imread(impath)
  # convert the image to grayscale
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  imsz = gray.shape[0]
  # derotate
  img=derotate(gray)

  # segment the image and write to file
  for rowind in range(9):
      for colind in range(9):
        subimg=img[(rowind*int(imsz/9)):((rowind+1)*int(imsz/9)),
                  (colind*int(imsz/9)):((colind+1)*int(imsz/9))]
        # convert to PIL and write to file
        # NOTE: converting to PIL to match the generating script
        Image.fromarray(subimg).save(f'{home_dir}{tmpdir}{rowind}{colind}.png')

  test_imgs=get_image_files(f'{home_dir}{tmpdir}')
  # sorting alphabetically works due to the naming structure in the loop above
  test_imgs.sort()  

  n_tta=10
  attempt_cntr=0
  failedsolve=True
  while failedsolve:
    attempt_cntr+=1

    tst_dl=learn.dls.test_dl(test_imgs)

    learn.epoch=0
    preds = learn.tta(dl=tst_dl,n=n_tta)[0]

    ### Attempt straightforward solution ###
    curgrid=pd.Series(torch.argmax(preds,dim=1).numpy()).map(labeldict)
    curgrid=''.join(curgrid)

    try:
      solved=solve(curgrid)
      solved_grid=''.join(solved.values())

      # failedsolve = False          
      return solved_grid
    except:
      print('Solving failed for '+impath)
      n_tta+=2
    
    if attempt_cntr>=5:
      print('Giving up on '+impath)
      return 'sudontku'

In [25]:
datestr=datetime.datetime.now(pytz.timezone('US/Eastern')).strftime("%y%m%d_%H%M")

solved_df=[]
solvdict={'id':None,'solved_grid':None}
for ind,impath in enumerate(imgs):
    print(f'{ind} / {len(imgs)}')

    imid=os.path.basename(impath)[:-4]
    print(imid)

    solvdict['id']=imid
    solvdict['solved_grid']=extractNsolve(impath)

    solved_df.append(solvdict.copy())
    
    if ind%100==0:
        outdf=pd.DataFrame(solved_df)
        outdf.to_pickle(f'/storage/BPI/Data/Sudoku/{datestr}_testsols.pkl')

#   clear_output()

outdf=pd.DataFrame(solved_df)
outdf.to_pickle(f'/storage/BPI/Data/Sudoku/{datestr}_testsols.pkl')

0 / 200
0536


1 / 200
1615


2 / 200
0041


3 / 200
2725


4 / 200
1189


...

195 / 200
4579


196 / 200
3080


197 / 200
2640


198 / 200
3144


199 / 200
2735


In [ ]:
tempdf=pd.DataFrame(solved_df)

In [ ]:
ind=2200
ind%100

In [ ]:
tempdf.head()

In [ ]:
np.any(tempdf['solved_grid']=='sudontku')

In [ ]:
# ind=17
# sudoku_display(tempdf.loc[ind,'solved_grid'])
# Image.open(imgs[ind])